# LangChain Expression Language (LCEL)

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole, chat_completion

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_gigachat import GigaChat
from langchain.schema.output_parser import StrOutputParser

## Простейшая цепочка (Simple Chain)

In [7]:
# ChatPromptTemplate - класс с объектами шаблонов для чата с языковой моделью
prompt = ChatPromptTemplate.from_template(
    "Расскажи короткую шутку о {topic}"
)
model = GigaChat(credentials=api_key, verify_ssl_certs=False)
output_parser = StrOutputParser()

In [15]:
chain = prompt | model | output_parser

In [16]:
chain.invoke({"topic": "медведи"})

'Почему медведь не может играть в прятки?\n because he always leaves his mark! 🐻'

## Более сложная цепочка (More complex chain)

**`RunnableMap`** — это структура в LangChain, позволяющая передавать данные через **словарь функций**, где каждая функция применяется к входным данным.

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

C:\Users\Maksim\AppData\Local\Temp\ipykernel_15480\554499278.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
c:\Users\Maksim\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Maksim\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to ef

In [29]:
"""
DocArrayInMemorySearch - простейшая векторная БД, 
разворачиваемая внутри программы, без необходимости во внешних сервисах.
"""
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Харрисон работал в хлеву", "медведи любят есть мед"],
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()

C:\Users\Maksim\AppData\Roaming\Python\Python311\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [30]:
retriever.get_relevant_documents("Где работал Харрисон?")

C:\Users\Maksim\AppData\Local\Temp\ipykernel_10072\2773580541.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Где работал Харрисон?")


[Document(metadata={}, page_content='Харрисон работал в хлеву'),
 Document(metadata={}, page_content='медведи любят есть мед')]

In [31]:
retriever.get_relevant_documents("Что медведям нравится есть")

[Document(metadata={}, page_content='медведи любят есть мед'),
 Document(metadata={}, page_content='Харрисон работал в хлеву')]

In [32]:
template = """Ответь на вопрос ориентируясь только на представленный далее контекст:
{context}

Вопрос: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain.schema.runnable import RunnableMap

In [34]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [40]:
chain.invoke({"question": "Где работал Харрисон?"})

'Харрисон работал в хлеву.'

In [43]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [44]:
chain.invoke({"question": "Где работал Харрисон?"})

'Харрисон работал в хлеву.'

In [36]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [37]:
inputs.invoke({"question": "Где работал Харрисон?"})

{'context': [Document(metadata={}, page_content='Харрисон работал в хлеву'),
  Document(metadata={}, page_content='медведи любят есть мед')],
 'question': 'Где работал Харрисон?'}

## Bind

Метод **bind** позволяет конфигурировать конкретную модель

In [14]:
functions = [
    {
      "name": "weather_search",
      "description": "Узнай погоду по коду аэропорта",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "Код аэропорта для узнавания там погоды"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = GigaChat(credentials=api_key, verify_ssl_certs=False, temperature=0)\
        .bind(functions=functions)

In [17]:
runnable = prompt | model

In [18]:
runnable.invoke({"input": "Какая погода в Санкт-Петербурге"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '8846a174-cb73-4b27-a247-113372c4a654'}, response_metadata={'token_usage': {'prompt_tokens': 83, 'completion_tokens': 29, 'total_tokens': 112}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-0586b5fb-6a98-415b-8d54-2c92ce571961-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'LED'}, 'id': '67f3f1d3-701c-4596-b7d8-602554898b89', 'type': 'tool_call'}])

In [54]:
functions = [
    {
      "name": "weather_search",
      "description": "Узнай погоду по коду аэропорта",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "Код аэропорта для узнавания там погоды"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Поиск новостей о последних спортивных событиях",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "Название спортивной команды для поиска"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [55]:
model = model.bind(functions=functions)

In [56]:
runnable = prompt | model

In [ ]:
"""
Модель "вызвала" функцию, которая на самом деле ничего не возвращает
Модель выдала ответ на основе своих знаний, это называется Hallucinated Tool Call
"""
runnable.invoke({"input": "Что делал футбольная команда из СПБ вчера?"})

NameError: name 'runnable' is not defined

In [60]:
"""
Модель "решила", что ей не нужно обращаться к функциям
Поэтому она сразу дала ответ
"""
runnable.invoke({"input": "Что делал Зенит вчера?"})

AIMessage(content='К сожалению, я не могу предоставить информацию о действиях конкретного футбольного клуба "Зенит" в реальном времени, так как у меня нет доступа к интернету и актуальным данным. Рекомендую проверить новостные сайты или официальные источники футбольного клуба "Зенит", чтобы узнать последние новости и результаты матчей.', additional_kwargs={'functions_state_id': '1e962837-8277-4802-816b-622555a96b4d'}, response_metadata={'token_usage': {'prompt_tokens': 142, 'completion_tokens': 72, 'total_tokens': 214}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-2c5c64d5-be89-42ce-94d4-003d5e050b51-0')

In [58]:
runnable.invoke({"input": "Какая погода в Санкт-Петербурге"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': 'b5aa160d-1470-48dd-b0a7-e7df97e64fd8'}, response_metadata={'token_usage': {'prompt_tokens': 144, 'completion_tokens': 29, 'total_tokens': 173}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-dfa59078-1446-4fc3-b7a4-a84b5aa7650c-0', tool_calls=[{'name': 'weather_search', 'args': {'airport_code': 'LED'}, 'id': 'cf2c9db9-7a81-40b4-b2bf-917b4654b128', 'type': 'tool_call'}])

## Fallbacks

In [64]:
import json

model = GigaChat(credentials=api_key, verify_ssl_certs=False)

In [77]:
simple_chain = model | json.loads

In [102]:
challenge = "Напишите три стихотворения в виде файла json, где каждое стихотворение представляет собой двоичный файл json с названием, \
             автором и первой строкой. Весь JSON должен находится тегами <JSON>.\
            \
            Пример:\
            <JSON>\
                {\
                    \"poems\": [\
                        {\
                        \"title\": \"Одинокая звезда\",\
                        \"author\": \"Александр Сергеевич Пушкин\",\
                        \"first_line\": \"Светит одна звезда в ночной тиши...\"\
                        },\
                        {\
                        \"title\": \"Ветер свободы\",\
                        \"author\": \"Анна Ахматова\",\
                        \"first_line\": \"Ветер свободы поет над полями...\"\
                        },\
                        {\
                        \"title\": \"Шёпот листьев\",\
                        \"author\": \"Федор Иванович Тютчев\",\
                        \"first_line\": \"Шёпот, робкое дыханье...\"\
                        }\
                    ]\
                }\
            <\JSON>\
            \
            Твой ответ:\
            <JSON>\
"

In [103]:
response = model.invoke(challenge)

In [104]:
response

AIMessage(content='К сожалению, я не могу создать JSON-файлы без использования дополнительных навыков. Если у вас есть конкретные требования к содержимому этих файлов (например, сами стихи), пожалуйста, предоставьте их, и я помогу вам сгенерировать нужный JSON.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 242, 'completion_tokens': 61, 'total_tokens': 303}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-dcc35b3d-ef9c-44ff-91e9-0d1e2e57e6fb-0')

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [69]:
simple_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

In [81]:
chain = model | StrOutputParser() | json.loads

In [82]:
chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Вообще, по хорошему, модель если бы выдала JSON (а я не смог её заставить это сделать, как ни пытался. Точнее, она либо выдавала его в не лучшем форматировании, с лишним текстом, либо вообще отказывалась генерировать его), дальнейшие действия цепочки распрарсили бы этот JSON

In [72]:
final_chain = simple_chain.with_fallbacks([chain])

In [73]:
final_chain.invoke(challenge)

TypeError: the JSON object must be str, bytes or bytearray, not AIMessage

## Interface

In [8]:
prompt = ChatPromptTemplate.from_template(
    "Расскажи мне короткую шутку о {topic}"
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [9]:
chain.invoke({"topic": "медведях"})

'Почему медведь сидит в углу?\n\nПотому что его баскетбольная команда играет плохо! 😄'

In [10]:
chain.batch([{"topic": "медведях"}, {"topic": "лягушках"}])

['Почему медведь сидит в углу?\n Потому что он уже спёр все сэндвичи! 😄',
 'Почему лягушки всегда такие счастливые?  \n Потому что у них нет лица, чтобы хмуриться!']

In [11]:
for t in chain.stream({"topic": "медведях"}):
    print(t)

Почему медведь сидит в углу?


Потому что его баскетбольная команда играет плохо!
 😄



In [12]:
response = await chain.ainvoke({"topic": "медведях"})
response

'Почему медведь сидит в углу?\n\nПотому что его баскетбольная команда играет сегодня! 😄'